#Install libraries

In [1]:
!pip install -q google-adk google-genai python-dotenv


#Configure Gemini and sanity check

In [2]:
import os
from google import genai

os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

client = genai.Client(api_key=GEMINI_API_KEY)

resp = client.models.generate_content(
    model="gemini-2.0-flash-001",
    contents="Say hello as a code review assistant.",
)
print(resp.text)


Hello! I'm here to assist with your code review. Let's take a look at the code and see how we can improve it.  What would you like me to focus on today?  For example:

*   Readability?
*   Performance?
*   Security?
*   Specific areas of concern?

Let's get started!



#Code review tool

In [3]:
from google.adk.tools import FunctionTool

def review_code(code: str, language: str, focus: str) -> str:
    """
    Use Gemini to review code.
    - code: the source code
    - language: e.g. 'python', 'javascript'
    - focus: e.g. 'readability', 'security', 'performance'
    """
    prompt = (
        f"You are a senior {language} code reviewer.\n\n"
        f"Review the following code with a focus on: {focus}.\n\n"
        "Code:\n"
        "```"
        f"{language}\n"
        f"{code}\n"
        "```\n\n"
        "Provide:\n"
        "1. A short summary.\n"
        "2. Specific issues.\n"
        "3. Concrete suggestions or improved snippets.\n"
    )

    resp = client.models.generate_content(
        model="gemini-2.0-flash-001",
        contents=prompt,
    )
    return resp.text.strip()

review_code_tool = FunctionTool(func=review_code)


/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


In [4]:
sample_code = "def add(a, b):\n    return a + b"
print(review_code(sample_code, "python", "readability and potential bugs"))


```python
def add(a, b):
    return a + b
```

1. **Summary:**

The code defines a simple function `add` that takes two arguments, `a` and `b`, and returns their sum. While functionally correct for basic numerical addition, it lacks type hinting and could benefit from a docstring to clarify its purpose and expected input types. It's crucial to consider how this function will behave when given non-numeric inputs.

2. **Specific Issues:**

*   **Lack of Type Hinting:** The code doesn't specify the expected data types for `a` and `b`. This reduces readability and makes it harder to catch potential type errors early on.
*   **Missing Docstring:**  The function lacks a docstring explaining its purpose, arguments, and return value. This makes the code less self-documenting.
*   **Potential Type Errors:** The `+` operator can behave differently depending on the types of `a` and `b`.  If they are strings, it will perform concatenation instead of addition.  The function doesn't handle or preven

#Code review agent

In [5]:
from google.adk.agents import LlmAgent

code_review_agent = LlmAgent(
    name="code_review_assistant",
    model="gemini-2.0-flash-001",
    instruction=(
        "You are a production-quality code review assistant.\n"
        "- The user will send you code and optionally what to focus on.\n"
        "- When appropriate, call the review_code tool to analyze the code.\n"
        "- Then give structured, actionable feedback: summary, issues, suggestions."
    ),
    tools=[review_code_tool],
)


#Runner and session

In [6]:
from google.adk.runners import InMemoryRunner

APP_NAME = "code_review_app"
USER_ID = "demo-review-user"

runner = InMemoryRunner(agent=code_review_agent, app_name=APP_NAME)

# Colab supports top-level await
session = await runner.session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
)

print("Session ID:", session.id)


Session ID: 6b774b21-4675-4ffa-973a-d36bc7ccdff2


#Helper to ask for a review

In [7]:
from google.genai.types import Content, Part

def ask_code_reviewer(code: str, language: str = "python", focus: str = "readability and potential bugs"):
    user_text = (
        f"Please review this {language} code.\n\n"
        f"Focus on: {focus}.\n\n"
        "Code:\n"
        "```"
        f"{language}\n"
        f"{code}\n"
        "```"
    )

    user_msg = Content(
        role="user",
        parts=[Part.from_text(text=user_text)],
    )

    print("=== USER REQUEST ===")
    print(user_text)
    print("\n=== REVIEW ASSISTANT RESPONSE ===")

    for event in runner.run(
        user_id=session.user_id,
        session_id=session.id,
        new_message=user_msg,
    ):
        if hasattr(event, "is_final_response") and event.is_final_response():
            if event.content:
                for part in event.content.parts:
                    text = getattr(part, "text", None)
                    if text:
                        print(text.strip())

# quick test
sample_code = "def add(a, b):\n    return a + b"
ask_code_reviewer(sample_code)


=== USER REQUEST ===
Please review this python code.

Focus on: readability and potential bugs.

Code:
```python
def add(a, b):
    return a + b
```

=== REVIEW ASSISTANT RESPONSE ===


Okay, I have reviewed the code. Here's my feedback:

**Summary:**

The code defines a simple addition function. While functionally correct, it lacks certain elements that would improve its readability, maintainability, and robustness.

**Issues:**

*   **Missing Type Hints:** The absence of type hints makes it difficult to immediately understand the expected input and output types.
*   **Missing Docstring:** There's no documentation explaining the function's purpose, arguments, or return value.
*   **Lack of Error Handling:** The code doesn't handle potential type errors if the inputs are not numbers.

**Suggestions:**

1.  **Add Type Hints:** Use type hints to specify the expected types for the input arguments and the return value. This improves readability and helps prevent type-related errors.
2.  **Add a Docstring:** Include a docstring to explain the function's purpose, arguments, and return value. This makes the code easier to understand and use.
3.  **Consider Error Handling:** 